## Creating a Logging Bucket

In [ ]:
import boto3
# print("Version: ", boto3.__version__)
# #list all buckets in s3 that the user can see
print("Buckets:", *boto3.client("s3").list_buckets()["Buckets"], sep="\n  ")
# #list current user's region
# print("Region: ", boto3.Session().region_name)

from botocore.exceptions import ClientError
import datetime
# import pandas as pd
# pyarrow must be installed to use parquet files in Pandas

import s3_delete as s3del
import s3_generate as s3gen
import s3_get as s3get
import s3_set as s3set

import display_modules as dp
# from inspect import getmembers, isfunction

%load_ext autoreload
%autoreload 2

- [Create](#create)
  - [Logging Bucket](#log)
- [Getters](#get)
- [Setters](#set)
  - [Lifecycle Management](#life)

Display the Functions available in the imported modules. Remember to use the import alias not the module name.

In [ ]:
for module in [s3del, s3gen, s3get, s3set]:
    dp.display_callables(module)

In [ ]:
s3_resource = boto3.resource("s3")
s3_client = boto3.client("s3")

<a id="create"></a>
## Create

<a id="log"></a>
### Create a Logging Bucket

Make the bucket that will be used to store the logs

In [ ]:
creator = ""
tags = s3gen.gen_tagging_list_from_python_dict({"creator":creator, "content":"logs"})
logging_bucket_name, logging_bucket_response = s3gen.gen_bucket("logging-bucket", tags=tags, suffix=True)
logging_bucket_response

Grant the logging bucket the required server access logging permissions via a policy

In [ ]:
source_account = ""
response = s3set.grant_logging_permissions_bucket_policy(logging_bucket_name, source_account)
response

Turn on logging for a source bucket using the newly created logging bucket as the target bucket

In [ ]:
source_bucket_name = ""
use_logging_bucket = ""
response = s3set.set_bucket_server_access_logging_on(source_bucket_name,
                                                     use_logging_bucket,
                                                     logging_path_prefix=None)
#the logging path prefix defaults to source_bucket_name if you keep it as None
response

<a id="get"></a>
## Get S3 Things

Get list of bucket dictionaries using a prefix and/or a date or date interval as a filter

In [ ]:
s3get.get_buckets_with_name_date("logging", [datetime.date(2023,2,12), "2023-2-16"])

Get list of bucket dictionaries using tags as filters

In [ ]:
s3get.get_buckets_with_tags({"fruit":"melon"}, s3_format=False)

Get list of object dictionaries using tags as filters

In [ ]:
s3get.get_objects_with_tags_from_bucket("melon", {"creator":"lemon"}, s3_format=False)

<a id="set"></a>
## Set S3 things

In [ ]:
bucket_name = "melon"
object_name = "lemon"
creator = "fruit-tree"

Add tags to a bucket

In [ ]:
tags = {"creator": creator, "content": "logs"}
s3set.add_tags_to_bucket(bucket_name, tags, s3_format=False)

Add tags to an object in a bucket

In [ ]:
tags = {"test-tag": "testingfunction_addtagtoobject"}
s3set.add_tags_to_object(bucket_name, object_name, tags, s3_format=False)

<a id="life"></a>
### Lifecycle Management Configuration

Adding lifecycle management configuration policies to the logging bucket made at the beginning

In [ ]:
bucket_lifecycle_config = ""
prefix_filter = ""
# tag_filter = {}
transition_response = s3set.add_bucket_lifecycle("logging-to-onezoneia-30days",
                           bucket_lifecycle_config,
                           transition="ONEZONE_IA",
                           transition_days=30,
                           expiration=False,
                           noncurrent_expiration=True,
                           noncurrent_expiration_days=3,
                           prefix_filter= prefix_filter)
expiration_response = s3set.add_bucket_lifecycle("logging-to-expiration-90days",
                           bucket_lifecycle_config,
                           transition=None,
                           expiration=True,
                           expiration_days=90,
                           noncurrent_expiration=True,
                           noncurrent_expiration_days=3,
                           prefix_filter=prefix_filter)

print("\nTransition Configuration:\n\t", transition_response)
print("\nExpiration Configuration:\n\t", expiration_response)